# Importieren der Libraries

In [1]:
import pandas as pd
import numpy as np
import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.svm import SVC
import xgboost as xgb

from sklearn.preprocessing import label_binarize

# Einlesen des Datensatzes

In [2]:
df = pd.read_csv("data/weatherHistory.csv")
df_tomorrow=df
df_tomorrow["Humidity"]= df_tomorrow["Humidity"]*100
df_tomorrow = df_tomorrow.drop(columns=['Precip Type', 
       'Apparent Temperature (C)','Wind Speed (km/h)',
       'Wind Bearing (degrees)', 'Visibility (km)', 'Loud Cover',
       'Pressure (millibars)'])
df_tomorrow['Timestamp'] = pd.to_datetime(df_tomorrow['Formatted Date'])

C:\Users\Katha\AppData\Local\Temp\ipykernel_7192\3276224646.py:8: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_tomorrow['Timestamp'] = pd.to_datetime(df_tomorrow['Formatted Date'])


# Vorhersage nächster Tag


## Zielvariablen erstellen

In [3]:
# Function to classify bad weather
#If Rain, then value 1 if not then value 0
def classify_bad_weather(summary):
    if 'rain' in summary.lower() or "dizzle" in summary.lower() or "storm" in summary.lower() or "windy" in summary.lower():
        return 1
    return 0

# Apply function to create new column
df_tomorrow['Bad_weather'] = df_tomorrow['Daily Summary'].apply(classify_bad_weather)

# Display the DataFrame
print(df_tomorrow["Bad_weather"].unique())

# Function to classify good weather
#If Rain, then value 1 if not then value 0
def classify_good_weather(summary):
    if 'clear' in summary.lower() or "partly cloudy" in summary.lower():
        return 1
    return 0

# Apply function to create new column
df_tomorrow['Good_weather'] = df_tomorrow['Daily Summary'].apply(classify_good_weather)

# Display the DataFrame
print(df_tomorrow["Good_weather"].unique())
df_tomorrow

# Function to classify rain
#If Rain, then value 1 if not then value 0
def classify_rain(summary):
    if 'rain' in summary.lower():
        return 1
    return 0

# Apply function to create new column
df_tomorrow['Rain'] = df_tomorrow['Daily Summary'].apply(classify_rain)

# Display the DataFrame
print(df_tomorrow["Rain"].unique())

# Funktion zur Klassifizierung
def classify_weather(row):
    if row['Good_weather'] == 1:
        return 0
    elif row['Bad_weather'] == 1:
        return 2
    else:
        return 1

# Neue Spalte basierend auf der Klassifizierungsfunktion erstellen
df_tomorrow['Weather'] = df_tomorrow.apply(classify_weather, axis=1)

# Ergebnis anzeigen
df_tomorrow

[0 1]
[1 0]
[0 1]


,Formatted Date,Summary,Temperature (C),Humidity,Daily Summary,Timestamp,Bad_weather,Good_weather,Rain,Weather
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,9.472222,89.0,Partly cloudy throughout the day.,2006-04-01 00:00:00+02:00,0,1,0,0
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,9.355556,86.0,Partly cloudy throughout the day.,2006-04-01 01:00:00+02:00,0,1,0,0
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,9.377778,89.0,Partly cloudy throughout the day.,2006-04-01 02:00:00+02:00,0,1,0,0
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,8.288889,83.0,Partly cloudy throughout the day.,2006-04-01 03:00:00+02:00,0,1,0,0
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,8.755556,83.0,Partly cloudy throughout the day.,2006-04-01 04:00:00+02:00,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
96448,2016-09-09 19:00:00.000 +0200,Partly Cloudy,26.016667,43.0,Partly cloudy starting in the morning.,2016-09-09 19:00:00+02:00,0,1,0,0
96449,2016-09-09 20:00:00.000 +0200,Partly Cloudy,24.583333,48.0,Partly cloudy starting in the morning.,2016-09-09 20:00:00+02:00,0,1,0,0
96450,2016-09-09 21:00:00.000 +0200,Partly Cloudy,22.038889,56.0,Partly cloudy starting in the morning.,2016-09-09 21:00:00+02:00,0,1,0,0
96451,2016-09-09 22:00:00.000 +0200,Partly Cloudy,21.522222,60.0,Partly cloudy starting in the morning.,2016-09-09 22:00:00+02:00,0,1,0,0


## Aufbereiten des Datensatzes

In [4]:
#Im datensatz fehlen der 02.04.2006-09.04.2006, deswegen wird der erste Tag abgeschnitten

df_naechster_tag = df_tomorrow.iloc[48:]
df_erster_tag =df_tomorrow.iloc[24:-24]
print(len(df_erster_tag))
print(len(df_naechster_tag))
df_erster_tag = df_erster_tag.reset_index()
df_naechster_tag = df_naechster_tag.reset_index()

df_erster_tag = df_erster_tag.drop(columns=["Formatted Date", "index",'Summary','Daily Summary','Timestamp',"Bad_weather", "Good_weather", "Rain", "Weather"])
#df_erster_tag = df_erster_tag.drop(columns=["Bad_weather", "Good_weather", "Rain", "Weather"])
df_erster_tag

96405
96405


,Temperature (C),Humidity
0,10.422222,62.0
1,9.911111,66.0
2,11.183333,80.0
3,7.155556,79.0
4,6.111111,82.0
...,...,...
96400,24.905556,51.0
96401,22.366667,58.0
96402,21.016667,64.0
96403,19.927778,71.0


In [5]:
df_tomorrow

,Formatted Date,Summary,Temperature (C),Humidity,Daily Summary,Timestamp,Bad_weather,Good_weather,Rain,Weather
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,9.472222,89.0,Partly cloudy throughout the day.,2006-04-01 00:00:00+02:00,0,1,0,0
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,9.355556,86.0,Partly cloudy throughout the day.,2006-04-01 01:00:00+02:00,0,1,0,0
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,9.377778,89.0,Partly cloudy throughout the day.,2006-04-01 02:00:00+02:00,0,1,0,0
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,8.288889,83.0,Partly cloudy throughout the day.,2006-04-01 03:00:00+02:00,0,1,0,0
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,8.755556,83.0,Partly cloudy throughout the day.,2006-04-01 04:00:00+02:00,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
96448,2016-09-09 19:00:00.000 +0200,Partly Cloudy,26.016667,43.0,Partly cloudy starting in the morning.,2016-09-09 19:00:00+02:00,0,1,0,0
96449,2016-09-09 20:00:00.000 +0200,Partly Cloudy,24.583333,48.0,Partly cloudy starting in the morning.,2016-09-09 20:00:00+02:00,0,1,0,0
96450,2016-09-09 21:00:00.000 +0200,Partly Cloudy,22.038889,56.0,Partly cloudy starting in the morning.,2016-09-09 21:00:00+02:00,0,1,0,0
96451,2016-09-09 22:00:00.000 +0200,Partly Cloudy,21.522222,60.0,Partly cloudy starting in the morning.,2016-09-09 22:00:00+02:00,0,1,0,0


## Visualisieren des Datensatzes

In [6]:
df_tomorrow.loc[df_tomorrow["Rain"]==1]

,Formatted Date,Summary,Temperature (C),Humidity,Daily Summary,Timestamp,Bad_weather,Good_weather,Rain,Weather
52680,2012-04-12 00:00:00.000 +0200,Mostly Cloudy,10.100000,61.0,Light rain in the morning.,2012-04-12 00:00:00+02:00,1,0,1,2
52681,2012-04-12 01:00:00.000 +0200,Partly Cloudy,10.183333,64.0,Light rain in the morning.,2012-04-12 01:00:00+02:00,1,0,1,2
52682,2012-04-12 02:00:00.000 +0200,Mostly Cloudy,10.350000,64.0,Light rain in the morning.,2012-04-12 02:00:00+02:00,1,0,1,2
52683,2012-04-12 03:00:00.000 +0200,Mostly Cloudy,10.700000,70.0,Light rain in the morning.,2012-04-12 03:00:00+02:00,1,0,1,2
52684,2012-04-12 04:00:00.000 +0200,Partly Cloudy,9.744444,73.0,Light rain in the morning.,2012-04-12 04:00:00+02:00,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...
95439,2016-10-26 19:00:00.000 +0200,Mostly Cloudy,10.555556,82.0,Rain until afternoon.,2016-10-26 19:00:00+02:00,1,0,1,2
95440,2016-10-26 20:00:00.000 +0200,Mostly Cloudy,9.733333,83.0,Rain until afternoon.,2016-10-26 20:00:00+02:00,1,0,1,2
95441,2016-10-26 21:00:00.000 +0200,Partly Cloudy,9.005556,83.0,Rain until afternoon.,2016-10-26 21:00:00+02:00,1,0,1,2
95442,2016-10-26 22:00:00.000 +0200,Clear,8.205556,83.0,Rain until afternoon.,2016-10-26 22:00:00+02:00,1,0,1,2


## Splitten in Test und Training

In [7]:
#Sort the dataframes by date
df_erster_tag = df_erster_tag.sort_index()
df_naechster_tag = df_naechster_tag.sort_index()

# Calculating the splitsizes
test_size = 0.3
split_index = int(len(df_erster_tag) * (1 - test_size))

# Splitting of the data
X_train = df_erster_tag.iloc[:split_index]
X_test = df_erster_tag.iloc[split_index:]
y_train = df_naechster_tag["Weather"].iloc[:split_index]
y_test = df_naechster_tag["Weather"].iloc[split_index:]

# Optional: Anzeigen der Größe der Daten
print(f"Trainingsdaten: {X_train.shape}, Trainingslabel: {y_train.shape}")
print(f"Testdaten: {X_test.shape}, Testlabel: {y_test.shape}")

Trainingsdaten: (67483, 2), Trainingslabel: (67483,)
Testdaten: (28922, 2), Testlabel: (28922,)


In [8]:
X_train

,Temperature (C),Humidity
0,10.422222,62.0
1,9.911111,66.0
2,11.183333,80.0
3,7.155556,79.0
4,6.111111,82.0
...,...,...
67478,31.016667,35.0
67479,29.994444,36.0
67480,28.050000,41.0
67481,27.844444,42.0


## Modelle Trainieren

### knn

In [9]:
# Initialisieren des k-NN-Klassifikators mit k=3
knn = KNeighborsClassifier(n_neighbors=3)

# Trainieren des Modells
knn.fit(X_train, y_train)

# Vorhersagen mit dem Testdatensatz
y_pred = knn.predict(X_test)

# Berechnen der Genauigkeit
accuracy = accuracy_score(y_test, y_pred)

# Berechnen des F1-Scores
f1 = f1_score(y_test, y_pred, average='weighted')  # Nutzen Sie 'weighted' für Multi-Klassen-Daten

# Berechnen der Konfusionsmatrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Berechnen des ROC-AUC-Scores
# Binarisieren der Labels für Multi-Klassen ROC-AUC
y_test_binarized = label_binarize(y_test, classes=np.unique(y_train))
y_pred_proba = knn.predict_proba(X_test)

if y_test_binarized.shape[1] == 1:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba[:, 1])
else:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, average='weighted', multi_class='ovr')

print("Vorhersagen:", y_pred)
print("Genauigkeit:", accuracy)
print("F1-Score:", f1)
print("Konfusionsmatrix:\n", conf_matrix)
print("ROC-AUC-Score:", roc_auc)


Vorhersagen: [1 0 1 ... 1 1 0]
Genauigkeit: 0.5875458128760114
F1-Score: 0.5913576041147499
Konfusionsmatrix:
 [[ 6062  4763     0]
 [ 6998 10931     0]
 [   77    91     0]]
ROC-AUC-Score: 0.6094607539356899


In [10]:
# Modell speichern
joblib.dump(knn, 'models/knn_tomorrow_model.pkl')

['models/knn_tomorrow_model.pkl']

### Random Forest

In [11]:
# Initialisieren des Random Forest-Klassifikators
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Trainieren des Modells
rf.fit(X_train, y_train)

# Vorhersagen mit dem Testdatensatz
y_pred = rf.predict(X_test)

# Berechnen der Genauigkeit
accuracy = accuracy_score(y_test, y_pred)

# Berechnen des F1-Scores
f1 = f1_score(y_test, y_pred, average='weighted')  # Nutzen Sie 'weighted' für Multi-Klassen-Daten

# Berechnen der Konfusionsmatrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Berechnen des ROC-AUC-Scores
# Binarisieren der Labels für Multi-Klassen ROC-AUC
y_test_binarized = label_binarize(y_test, classes=np.unique(y_train))
y_pred_proba = knn.predict_proba(X_test)

if y_test_binarized.shape[1] == 1:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba[:, 1])
else:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, average='weighted', multi_class='ovr')

print("Vorhersagen:", y_pred)
print("Genauigkeit:", accuracy)
print("F1-Score:", f1)
print("Konfusionsmatrix:\n", conf_matrix)
print("ROC-AUC-Score:", roc_auc)


Vorhersagen: [1 0 1 ... 1 0 0]
Genauigkeit: 0.5822211465320517
F1-Score: 0.5866947885457683
Konfusionsmatrix:
 [[ 5981  4821    23]
 [ 7028 10858    43]
 [   71    97     0]]
ROC-AUC-Score: 0.6094607539356899


In [12]:
# Modell speichern
joblib.dump(rf, 'models/rf_tomorrow_model.pkl')

['models/rf_tomorrow_model.pkl']

### XGB

In [13]:
# Initialisieren und Trainieren des XGBoost Modells
xgb_model = xgb.XGBClassifier(scale_pos_weight=len(y_train)/sum(y_train == 1))
xgb_model.fit(X_train, y_train)

# Vorhersagen mit dem Testdatensatz
y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)

# Berechnen der Genauigkeit
accuracy = accuracy_score(y_test, y_pred)

# Berechnen des F1-Scores
f1 = f1_score(y_test, y_pred, average='weighted')  # Nutzen Sie 'weighted' für Multi-Klassen-Daten

# Berechnen der Konfusionsmatrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Berechnen des ROC-AUC-Scores
# Binarisieren der Labels für Multi-Klassen ROC-AUC
y_test_binarized = label_binarize(y_test, classes=np.unique(y_train))
if y_test_binarized.shape[1] == 1:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba[:, 1])
else:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, average='weighted', multi_class='ovr')

print("Vorhersagen:", y_pred)
print("Genauigkeit:", accuracy)
print("F1-Score:", f1)
print("Konfusionsmatrix:\n", conf_matrix)
print("ROC-AUC-Score:", roc_auc)


c:\Users\Katha\anaconda3\envs\projekt_konzept\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:11:29] WARNING: C:\b\abs_0fh_d4x2ng\croot\xgboost-split_1713973188995\work\cpp_src\src\learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


Vorhersagen: [0 0 0 ... 0 0 0]
Genauigkeit: 0.6321831132010235
F1-Score: 0.6353933585808522
Konfusionsmatrix:
 [[ 6812  4011     2]
 [ 6457 11472     0]
 [   50   118     0]]
ROC-AUC-Score: 0.6818029919100833


In [14]:
# Modell speichern
joblib.dump(xgb_model, 'models/xgb_tomorrow_model.pkl')

['models/xgb_tomorrow_model.pkl']

### SVM

In [15]:
# Initialisieren des SVM-Klassifikators
svm = SVC(kernel='linear', random_state=42)

# Trainieren des Modells
svm.fit(X_train, y_train)

# Vorhersagen mit dem Testdatensatz
y_pred = svm.predict(X_test)

# Berechnen der Genauigkeit
accuracy = accuracy_score(y_test, y_pred)

# Berechnen des F1-Scores
f1 = f1_score(y_test, y_pred, average='weighted')

# Berechnen der Konfusionsmatrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Berechnen des ROC-AUC-Scores
# Binarisieren der Labels für Multi-Klassen ROC-AUC
y_test_binarized = label_binarize(y_test, classes=np.unique(y_train))
y_pred_proba = knn.predict_proba(X_test)

if y_test_binarized.shape[1] == 1:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba[:, 1])
else:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, average='weighted', multi_class='ovr')

print("Vorhersagen:", y_pred)
print("Genauigkeit:", accuracy)
print("F1-Score:", f1)
print("Konfusionsmatrix:\n", conf_matrix)
print("ROC-AUC-Score:", roc_auc)

Vorhersagen: [0 0 0 ... 0 0 0]
Genauigkeit: 0.6409307793375285
F1-Score: 0.6426810456818735
Konfusionsmatrix:
 [[ 6471  4354     0]
 [ 5863 12066     0]
 [   34   134     0]]
ROC-AUC-Score: 0.6094607539356899


In [16]:
# Modell speichern
joblib.dump(svm, 'models/svm_tomorrow_model.pkl')

['models/svm_tomorrow_model.pkl']

# Vorhersage Regen

## Aufbereiten des Datensatzes

In [17]:
df_rain = df
df_rain = df_rain.drop(columns=[ 'Formatted Date','Precip Type', 
       'Apparent Temperature (C)','Wind Speed (km/h)',
       'Wind Bearing (degrees)', 'Visibility (km)', 'Loud Cover',
       'Pressure (millibars)'])

# Given array
weather_array = np.array(df_rain["Summary"].unique())

# Filtering for elements that imply rain
rain_keywords = ['Rain', 'drizzle']
rain_descriptions = [desc for desc in weather_array if any(keyword in desc for keyword in rain_keywords)]

print(rain_descriptions)

['Light Rain', 'Rain']


In [18]:
# Function to classify rain
#If Rain, then value 1 if not then value 0
def classify_rain(summary):
    if 'rain' in summary.lower():
        return 1
    return 0

# Apply function to create new column
df_rain['Rain'] = df_rain['Summary'].apply(classify_rain)

# Display the DataFrame
print(df_rain["Rain"].unique())
df_rain = df_rain.drop(columns =["Summary", "Daily Summary"])


[0 1]


In [19]:
#Test Dataframe, if it is balanced. Since it is inbalanced, a imblearn algorithm will be used for Training and Testing
df_rain.groupby("Rain").count()

,Temperature (C),Humidity
Rain,,
0,96380,96380
1,73,73


In [20]:
df_rain.corr()

,Temperature (C),Humidity,Rain
Temperature (C),1.000000,-0.632255,-0.005477
Humidity,-0.632255,1.000000,0.022705
Rain,-0.005477,0.022705,1.000000


## Splitten des Datensatzes

In [21]:
#Sort the dataframes by date
df_rain = df_rain.sort_index()

# Calculating the splitsizes
test_size = 0.3
split_index = int(len(df_rain) * (1 - test_size))
df_rain_wo = df_rain.drop(columns = ["Rain"])
# Splitting of the data
X_train = df_rain_wo.iloc[:split_index]
X_test = df_rain_wo.iloc[split_index:]
y_train = df_rain["Rain"].iloc[:split_index]
y_test = df_rain["Rain"].iloc[split_index:]

# Optional: Anzeigen der Größe der Daten
print(f"Trainingsdaten: {X_train.shape}, Trainingslabel: {y_train.shape}")
print(f"Testdaten: {X_test.shape}, Testlabel: {y_test.shape}")

Trainingsdaten: (67517, 2), Trainingslabel: (67517,)
Testdaten: (28936, 2), Testlabel: (28936,)


## Trainieren der Modelle

### knn

In [22]:
# Initialisieren des k-NN-Klassifikators
knn = KNeighborsClassifier(n_neighbors=15)

# Trainieren des Modells
knn.fit(X_train, y_train)

# Vorhersagen mit dem Testdatensatz
y_pred = knn.predict(X_test)

# Berechnen der Genauigkeit
accuracy = accuracy_score(y_test, y_pred)

# Berechnen des F1-Scores
f1 = f1_score(y_test, y_pred, average='weighted')  # Nutzen Sie 'weighted' für Multi-Klassen-Daten

# Berechnen der Konfusionsmatrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Berechnen des ROC-AUC-Scores
# Binarisieren der Labels für Multi-Klassen ROC-AUC
y_test_binarized = label_binarize(y_test, classes=np.unique(y_train))
y_pred_proba = knn.predict_proba(X_test)

if y_test_binarized.shape[1] == 1:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba[:, 1])
else:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, average='weighted', multi_class='ovr')

print("Vorhersagen:", y_pred)
print("Genauigkeit:", accuracy)
print("F1-Score:", f1)
print("Konfusionsmatrix:\n", conf_matrix)
print("ROC-AUC-Score:", roc_auc)


Vorhersagen: [0 0 0 ... 0 0 0]
Genauigkeit: 0.9980646944982029
F1-Score: 0.9970979790060919
Konfusionsmatrix:
 [[28880     0]
 [   56     0]]
ROC-AUC-Score: 0.5251514889196677


In [23]:
# Modell speichern
joblib.dump(knn, 'models/knn_rain_model.pkl')

['models/knn_rain_model.pkl']

### Random Forest

In [24]:
# Initialisieren des Random Forest-Klassifikators
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Trainieren des Modells
rf.fit(X_train, y_train)

# Vorhersagen mit dem Testdatensatz
y_pred = rf.predict(X_test)

# Berechnen der Genauigkeit
accuracy = accuracy_score(y_test, y_pred)

# Berechnen des F1-Scores
f1 = f1_score(y_test, y_pred, average='weighted')  # Nutzen Sie 'weighted' für Multi-Klassen-Daten

# Berechnen der Konfusionsmatrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Berechnen des ROC-AUC-Scores
# Binarisieren der Labels für Multi-Klassen ROC-AUC
y_test_binarized = label_binarize(y_test, classes=np.unique(y_train))
y_pred_proba = knn.predict_proba(X_test)

if y_test_binarized.shape[1] == 1:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba[:, 1])
else:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, average='weighted', multi_class='ovr')

print("Vorhersagen:", y_pred)
print("Genauigkeit:", accuracy)
print("F1-Score:", f1)
print("Konfusionsmatrix:\n", conf_matrix)
print("ROC-AUC-Score:", roc_auc)

Vorhersagen: [0 0 0 ... 0 0 0]
Genauigkeit: 0.9978918993641139
F1-Score: 0.9970115740476937
Konfusionsmatrix:
 [[28875     5]
 [   56     0]]
ROC-AUC-Score: 0.5251514889196677


In [25]:
# Modell speichern
joblib.dump(rf, 'models/rf_rain_model.pkl')

['models/rf_rain_model.pkl']

### xgb

In [26]:
# Initialisieren und Trainieren des XGBoost Modells
xgb_model = xgb.XGBClassifier(scale_pos_weight=len(y_train)/sum(y_train == 1))
xgb_model.fit(X_train, y_train)

# Vorhersagen mit dem Testdatensatz
y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# Berechnen der Genauigkeit
accuracy = accuracy_score(y_test, y_pred)

# Berechnen des F1-Scores
f1 = f1_score(y_test, y_pred, average='weighted')  # Nutzen Sie 'weighted' für Multi-Klassen-Daten

# Berechnen der Konfusionsmatrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Berechnen des ROC-AUC-Scores
# Binarisieren der Labels für Multi-Klassen ROC-AUC
y_test_binarized = label_binarize(y_test, classes=np.unique(y_train))
if y_test_binarized.shape[1] == 1:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba)
else:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, average='weighted', multi_class='ovr')

print("Vorhersagen:", y_pred)
print("Genauigkeit:", accuracy)
print("F1-Score:", f1)
print("Konfusionsmatrix:\n", conf_matrix)
print("ROC-AUC-Score:", roc_auc)

Vorhersagen: [0 0 0 ... 0 0 0]
Genauigkeit: 0.9960257119159525
F1-Score: 0.9961423618885212
Konfusionsmatrix:
 [[28819    61]
 [   54     2]]
ROC-AUC-Score: 0.7688214162049861


In [27]:
# Modell speichern
joblib.dump(xgb_model, 'models/xgb_rain_model.pkl')

['models/xgb_rain_model.pkl']

### SVM

In [28]:
# Initialisieren des SVM-Klassifikators
svm = SVC(kernel='linear', random_state=42)

# Trainieren des Modells
svm.fit(X_train, y_train)

# Vorhersagen mit dem Testdatensatz
y_pred = svm.predict(X_test)

# Berechnen der Genauigkeit
accuracy = accuracy_score(y_test, y_pred)

# Berechnen des F1-Scores
f1 = f1_score(y_test, y_pred, average='weighted')

# Berechnen der Konfusionsmatrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Berechnen des ROC-AUC-Scores
# Binarisieren der Labels für Multi-Klassen ROC-AUC
y_test_binarized = label_binarize(y_test, classes=np.unique(y_train))
y_pred_proba = knn.predict_proba(X_test)

if y_test_binarized.shape[1] == 1:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba[:, 1])
else:
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, average='weighted', multi_class='ovr')

print("Vorhersagen:", y_pred)
print("Genauigkeit:", accuracy)
print("F1-Score:", f1)
print("Konfusionsmatrix:\n", conf_matrix)
print("ROC-AUC-Score:", roc_auc)

Vorhersagen: [0 0 0 ... 0 0 0]
Genauigkeit: 0.9980646944982029
F1-Score: 0.9970979790060919
Konfusionsmatrix:
 [[28880     0]
 [   56     0]]
ROC-AUC-Score: 0.5251514889196677


In [29]:
# Modell speichern
joblib.dump(svm, 'models/svm_rain_model.pkl')

['models/svm_rain_model.pkl']